In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import time
import os, requests

In [21]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(r"D:\GS_WebScarping\project_08_mag\chromedriver.exe")
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    
    
     # clck login
    try:
        # log_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="site-nav__account"]'))) 
        login = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "Login"]')))
        if login:
            login.click()
    except Exception as e:
        print("no login")

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    
    # get password tab
    try:
        provided_pass = "Sultanmirza1501#"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    
    # get the check done
    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
            # driver.execute_script("arguments[0].scrollIntoView();", check)
            check.click()
            time.sleep(2)
            for i in range(2):
                driver.back()
    except Exception as e:
        print(f"No check tab found and the error is {e}") 
    
     # number of cars
    try:
        # time.sleep(1)
        cars = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/span[@class="sort-page"]'))).text.strip()
        if cars:
            print(cars)
            splited_cars = cars.split(" ")
            total_cars = int(splited_cars[4])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
    
    
    # result
    results = []
    car_count = 0
    while car_count < 2: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card product-card"]')))
            
            for i in range(2): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card product-card"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}
                    
                    # lot_num
                    try:
                        lot = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/span[@class="pill-item pill-item-lot"]'))).text.strip()
                        if lot:
                            details['Lot'] = lot
                        else:
                            details['Lot'] = 'na'
                    except Exception as e:
                        print("No lot values found")
                    # veh cont --> done
                    try:
                        cont_card =WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="content-section content-section-white content-section-details"]'))) 
                        if cont_card:
                            # title and inspec repo --> done
                            try:
                                cont_head =WebDriverWait(cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="section-header"]')))  
                                if cont_head:
                                    title = cont_head.find_element(By.XPATH, './/div[@class="header-title"]').text.strip()
                                    if title:
                                        details['Title'] = title.split("\n")
                                    else:
                                        details['Title'] = 'na'
                                    
                                    inspec = cont_head.find_element(By.XPATH, './/a[text()="View PDF"]').get_attribute("href")
                                    if inspec:
                                        details['Inspection'] = inspec
                                    else:
                                        details['Inspection'] = 'na'
                                        
                            except Exception as e:
                                print("No cont head")
                            
                            # vendor  --> done
                            try:
                                vend =WebDriverWait(cont_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'strong'))).text.strip()
                                if vend:
                                    splited_vend = vend.split(": ")
                                    details[splited_vend[0]] = splited_vend[1]
                                # else:
                            except Exception as e:
                                print("No vendor")
                            
                            # other det --> done
                            try:
                                cont_ul = WebDriverWait(cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/ul[@class="details-list"]')))
                                if cont_ul:
                                    cont_items = cont_ul.find_elements(By.XPATH, './/li[@class="detail-item"]')
                                    if cont_items:
                                        for cont_item in cont_items:
                                            cont_lbl = cont_item.find_element(By.TAG_NAME, 'span').text.strip()
                                            cont_val = cont_item.find_element(By.TAG_NAME, 'strong').text.strip()
                                            if cont_lbl and cont_val:
                                                details[cont_lbl] = cont_val
                                            else:
                                                print("No cont info found")
                                    else:
                                        print("No cont items")
                                else:
                                    print("No cont ul")
                            except Exception as e:
                                print("No cont info found")
                            
                            # features --> done
                            try:
                                fet =WebDriverWait(cont_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="mt-4"]'))).text.strip()
                                if fet:
                                    details['Features'] = fet
                                else:
                                    details['Features'] = 'na'
                            except Exception as e:
                                print("No car features")  
                        else:
                            print("No cont header")
                    except Exception as e:
                        print("No veh cont information")
                    
                    
                    # images --> done
                    try:
                        imgs =WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@alt="Vehicle image"]'))) 
                        if imgs:
                            imgs_list = [img.get_attribute('src') for img in imgs]
                            details['Images'] = ", ".join(imgs_list)
                        else:
                            print("No images found")
                        
                    except Exception as e:
                        print("No images")
                    
                    
                    #details --> done
                    try:
                        car_det_card = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="economy-desktop"]')))
                        if car_det_card:
                            det_ul = car_det_card.find_element(By.XPATH, './/ul[@class="tab-grid details-list"]')
                            if det_ul:
                                det_items = det_ul.find_elements(By.XPATH, './/li[@class="detail-item"]')
                                for det_item in det_items:
                                    det_lbl = det_item.find_element(By.TAG_NAME, 'span').text.strip()
                                    det_val = det_item.find_element(By.TAG_NAME, 'strong').text.strip()
                                    if det_lbl and det_val:
                                        details[det_lbl]  = det_val
                                    else:
                                        print("No car details")            
                    except Exception as e:
                        print("No car details")
                        
                    
                    # mechanical
                    try:
                        mech_main = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="mechanical"]')))
                        if mech_main:
                            
                            # interior
                            try:
                                interior = WebDriverWait(mech_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//ul[@class="details-list"])[1]')))
                                if interior:
                                    int_items = interior.find_elements(By.XPATH, './/li[@class="detail-item"]')
                                    for int_item in int_items:
                                        int_lbl = int_item.find_element(By.TAG_NAME, 'span').text.strip()
                                        int_val = int_item.find_element(By.TAG_NAME, 'strong').text.strip()
                                        if int_lbl and int_val:
                                            details[int_lbl]  = int_val
                                        else:
                                            print("No car interior details")
                            except Exception as e:
                                print("No car interior info")
                                
                            # tyres
                            try:
                                tyres = WebDriverWait(mech_main, 5).until(EC.presence_of_element_located((By.XPATH, '(.//ul[@class="details-list"])[2]')))
                                if tyres:
                                    tyre_items = tyres.find_elements(By.XPATH, './/li[@class="detail-item"]')
                                    for tyre_item in tyre_items:
                                        tyre_lbl = tyre_item.find_element(By.TAG_NAME, 'span').text.strip()
                                        tyre_val = tyre_item.find_element(By.TAG_NAME, 'strong').text.strip()
                                        if tyre_lbl and tyre_val:
                                            details[tyre_lbl]  = tyre_val
                                        else:
                                            print("No car tyres details")
                            except Exception as e:
                                print("No car tyres info")
                            
                            
                            # damage images  --> done
                            try:
                                dmg_imgs =WebDriverWait(mech_main, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="img-responsive largeimage"]'))) 
                                if dmg_imgs:
                                    dmg_imgs_list = [dmg_img.get_attribute('src') for dmg_img in dmg_imgs]
                                    details['Dmg_Images'] = ", ".join(dmg_imgs_list)
                                else:
                                    print("No damage images found")
                                
                            except Exception as e: 
                                print("No damage images found")
                            
                            # damage_text
                            try:
                                dmg_texts = WebDriverWait(mech_main, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/img[@class="img-responsive largeimage"]')))
                                if dmg_texts:
                                    dmg_text_list = [dmg_text.get_attribute("alt") for dmg_text in dmg_texts]
                                    details['Dmg_text'] = ", ".join(dmg_text_list)
                                else:
                                    print("No damage texts found")
                                
                            except Exception as e: 
                                print("No damage texts found")
                    except Exception as e:
                        print("No car mech information")
                    car_count+=1
                    results.append(details)
                    driver.back()   
                
                except Exception as e:
                    print("No link to click on")   
                
            if car_count % 30 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                try:
                    next_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, './/li[@class="page-item page-item-arrow page-item-arrow-next"]')))
                    if next_link:
                        driver.execute_script("arguments[0].scrollIntoView();", next_link)
                        time.sleep(2)
                        next_link.click()
                        
                        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="card product-card"]')))
                        print("Hit the next button")
                except Exception as e:
                    print("No next found")
                    break
        except Exception as e:
            print("No more car links to click")
            break
    
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("fleet.csv", index=False)
    driver.quit()
    
path = 'https://www.fleetauctiongroup.com/auction/172'
scarpe(path)

1 - 30 of 62 Results
62 number of cars found
No car features
No car tyres details
No car features


In [23]:
df = pd.read_csv("fleet.csv")
df.columns

Index(['Lot', 'Title', 'Inspection', 'Vendor', 'Registration', 'Registered',
       'Miles', 'Miles Warranted', 'Former Keepers', 'Fuel', 'Transmission',
       'Colour', 'VAT', 'MOT', 'V5', 'Keys', 'CAP Clean', 'CC', 'CAP Average',
       'CAP Below', 'Images', 'Service history', 'Service documents',
       'Number of stamps', 'Last service date', 'Last service miles',
       'MOT Due', 'Body Style', 'Hubs', 'BHP', 'Gross Weight', 'Euro Status',
       'Euro', 'CO2', 'MPG Combined', 'MPG Urban', 'MPG Extra Urban',
       'Carpet condition', 'Seat condition', 'Upholstery condition', 'Odour',
       'Tyre NSF', 'Tyre OSF', 'Tyre NSR', 'Tyre OSR', 'Dmg_Images',
       'Dmg_text', 'Spare Tyre'],
      dtype='object')

In [24]:
df['Dmg_text'][1]

'C Post os Dented 2 Or Less UpTo 10mm, Wheel osf Scratched Over 50mm, Wing osf Chipped 1 To 5, Door osf Dented Over 30mm, Qtr Panel Trim os Scuffed Over 25mm, Rear Panel Trim Int Scuffed Over 25mm, Rear Panel Trim Int Scuffed Over 25mm, Carpets Front Soiled Light, Door nsr Dented 2 Or Less UpTo 10mm, Bumper Front Chipped 1 To 5, Bonnet Scratched Over 25mm Thru Paint'

In [4]:
reg_img = df[['Registration', 'Images']]

In [6]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Registration"] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)

Downloaded: Images\GD21BWH\GD21BWH_1_00_001_GD21BWH_ext---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_2_00_002_GD21BWH_ext---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_3_00_003_GD21BWH_ext---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_4_00_004_GD21BWH_ext---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_5_00_005_GD21BWH_ext---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_6_00_006_GD21BWH_ext---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_7_00_007_GD21BWH_ext---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_8_00_008_GD21BWH_ext---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_9_01-2141983-131411---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_10_02-2141975-131411---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_11_03-2141973-131411---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_12_04-2141969-131411---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_13_05-2141974-131411---1024-768.jpg
Downloaded: Images\GD21BWH\GD21BWH_14_05-2141982-131411---1024-768.jpg
Downloa

KeyboardInterrupt: 

In [25]:
cond_imgs = df[['Dmg_text', 'Dmg_Images', 'Registration']]
cond_imgs

,Dmg_text,Dmg_Images,Registration
0,"Bonnet Chipped 1 To 5, Wheel nsf Scratched Ove...",https://dgaww6lqj3.execute-api.eu-west-1.amazo...,AF65OZU
1,"C Post os Dented 2 Or Less UpTo 10mm, Wheel os...",https://dgaww6lqj3.execute-api.eu-west-1.amazo...,RV71UKE


In [26]:
def download_images(data, main_folder="Damage"):  # Main folder is now 'Damage'
    os.makedirs(main_folder, exist_ok=True)

    for index, row in data.iterrows():
        reg_no = row["Registration"]
        image_urls = row["Dmg_Images"].split(", ")  # Splitting multiple URLs
        dmg_texts = row["Dmg_text"].split(", ")  # Splitting corresponding texts

        reg_folder = os.path.join(main_folder, reg_no)  
        os.makedirs(reg_folder, exist_ok=True)  

        for idx, (url, dmg_text) in enumerate(zip(image_urls, dmg_texts)):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)

            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue

            try:
                response = requests.get(url, stream=True)
                response.raise_for_status()

                file_extension = os.path.splitext(parsed_url.path)[-1].lower()
                if file_extension not in [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".webp"]:
                    file_extension = ".jpg"

                # Clean the damage text to make it a valid filename
                safe_dmg_text = "".join(c if c.isalnum() or c in " _-" else "_" for c in dmg_text).strip()

                full_file_name = os.path.join(reg_folder, f"{safe_dmg_text}_{idx + 1}_{file_extension}")

                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(cond_imgs)

Downloaded: Damage\AF65OZU\Bonnet Chipped 1 To 5_1_.jpg
Downloaded: Damage\AF65OZU\Wheel nsf Scratched Over 50mm_2_.jpg
Downloaded: Damage\AF65OZU\Wheel nsr Scratched Over 50mm_3_.jpg
Downloaded: Damage\AF65OZU\Bumper Rear Scratched Up To 25mm Thru Paint_4_.jpg
Downloaded: Damage\AF65OZU\Wheel osr Scratched Over 50mm_5_.jpg
Downloaded: Damage\AF65OZU\Door osf Chipped Edge Chip_6_.jpg
Downloaded: Damage\RV71UKE\C Post os Dented 2 Or Less UpTo 10mm_1_.jpg
Downloaded: Damage\RV71UKE\Wheel osf Scratched Over 50mm_2_.jpg
Downloaded: Damage\RV71UKE\Wing osf Chipped 1 To 5_3_.jpg
Downloaded: Damage\RV71UKE\Door osf Dented Over 30mm_4_.jpg
Downloaded: Damage\RV71UKE\Qtr Panel Trim os Scuffed Over 25mm_5_.jpg
Downloaded: Damage\RV71UKE\Rear Panel Trim Int Scuffed Over 25mm_6_.jpg
Downloaded: Damage\RV71UKE\Rear Panel Trim Int Scuffed Over 25mm_7_.jpg
Downloaded: Damage\RV71UKE\Carpets Front Soiled Light_8_.jpg
Downloaded: Damage\RV71UKE\Door nsr Dented 2 Or Less UpTo 10mm_9_.jpg
Downloaded: Dam